In [2]:
## Dependencies ##
#!pip3 install python-catmaid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
from scipy.stats import kstest, pearsonr, mannwhitneyu, hmean, iqr
import seaborn as sns
import pymaid
import navis

sns.set_style("ticks")
sns.set_palette("colorblind")

# Wild Type - Seymour

In [3]:
# Load neurons from Wild Type volumes
navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
wt = pymaid.connect_catmaid(server="https://l1em.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 

# Get connectors of neurons of interest
print(f'Server is running CATMAID version {wt.catmaid_version}')
neurons_wt = pymaid.find_neurons(annotations='Valdes-Aleman et al. 2021')
connectors_wt = pymaid.get_connectors_between(neurons_wt, neurons_wt)
connectors_wt 

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


INFO  : Found 37 neurons matching the search parameters (pymaid)                


connector_id                 connector_loc  node1_id  source_neuron  \
0           478360  [63699.4, 83474.6, 119700.0]    478355          11995   
1           988063  [61681.6, 85329.0, 118950.0]    988060          11995   
2           988063  [61681.6, 85329.0, 118950.0]    988060          11995   
3          3543174  [46059.8, 83429.0, 114650.0]   3543194          11995   
4          4420609  [48474.7, 81430.2, 129950.0]   4420615          11995   
...            ...                           ...       ...            ...   
1416       7168246  [48130.8, 83043.3, 128300.0]   7168252       10804041   
1417      10812818  [66956.0, 87392.4, 129850.0]  10812793       10804041   
1418      12564988  [45216.2, 81931.8, 127250.0]  10816842       10804041   
1419      12997665  [61677.8, 84724.8, 129250.0]  10804942       10804041   
1420      20376232  [61727.0, 85158.1, 128150.0]  10805039       10804041   

      confidence1 creator1                     node1_loc  node2_id  \
0               5      NaN  [63809.6, 83676.0, 119700.0]    478346   
1               5      NaN  [61810.8, 85416.4, 118950.0]    988085   
2               5      NaN  [61810.8, 85416.4, 118950.0]    988069   
3               5      NaN  [46002.8, 83516.4, 114650.0]   3543173   
4               5      NaN  [48343.6, 81299.1, 129950.0]   4420607   
...           ...      ...                           ...       ...   
1416            5      NaN  [48184.0, 83345.4, 128300.0]   6290267   
1417            5      NaN  [66774.5, 87495.0, 129850.0]   7202182   
1418            5      NaN  [45252.9, 82021.7, 127250.0]   4413776   
1419            5      NaN  [61794.6, 84702.0, 129250.0]  12997662   
1420            5      NaN  [61804.1, 85218.8, 128150.0]   7595545   

      target_neuron  confidence2 creator2                     node2_loc  
0            471705            5      NaN  [63570.2, 83596.2, 119800.0]  
1           3034133            5      NaN  [61522.0, 85051.6, 118950.0]  
2           6207232            5      NaN  [61913.4, 85154.2, 118950.0]  
3           3282869            5      NaN  [46266.9, 83221.9, 114650.0]  
4           7599503            5      NaN  [48260.9, 81805.5, 130000.0]  
...             ...          ...      ...                           ...  
1416        3282869            5      NaN  [47871.4, 82904.1, 128250.0]  
1417        7195651            5      NaN  [66824.9, 87137.8, 129800.0]  
1418        7599503            5      NaN  [45079.4, 81994.5, 127300.0]  
1419        7599503            5      NaN  [61666.4, 85028.8, 129250.0]  
1420         471720            5      NaN  [61743.4, 84493.3, 128150.0]  

[1421 rows x 12 columns]

In [21]:
#Find connectors below 100nm

# Function to compute Euclidean distance
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

# Compute distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_wt.apply(lambda row: euclidean_distance(row['connector_loc'], row['node2_loc']), axis=1)
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
wt_filtered_df = connectors_wt[small_distances]
len(wt_filtered_df)

9

# TNT Volume

In [23]:
#TNT Volume# Connect to the VFB CATMAID server hosting the FAFB data
pymaid.clear_cache()
tnt = pymaid.connect_catmaid(server="https://iav-tnt.catmaid.virtualflybrain.org/", project_id=2, api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {tnt.catmaid_version}')
neurons_tnt = pymaid.find_neurons(annotations='Valdes-Aleman et al. 2021')

# Get connectors of neurons of interest
print(f'Server is running CATMAID version {wt.catmaid_version}')
connectors_tnt = pymaid.get_connectors_between(neurons_tnt, neurons_tnt)
connectors_tnt 

INFO  : Cached cleared. (pymaid)
INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


INFO  : Found 36 neurons matching the search parameters (pymaid)                
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


INFO  : Found 36 neurons matching the search parameters (pymaid)                


connector_id                connector_loc  node1_id  source_neuron  \
0        19272919  [57813.4, 26462.7, 15880.0]  19272920       19243093   
1        19272932  [57285.6, 26135.5, 15400.0]  19272933       19243093   
2        19272932  [57285.6, 26135.5, 15400.0]  19272933       19243093   
3        19272955  [57855.8, 26723.2, 15480.0]  19272956       19243093   
4        19272955  [57855.8, 26723.2, 15480.0]  19272956       19243093   
..            ...                          ...       ...            ...   
746      19287943   [53010.5, 24947.6, 6240.0]  19442479       19405635   
747      19442515   [53847.9, 24936.4, 6720.0]  19442516       19405635   
748      19442558   [53741.4, 24991.1, 8680.0]  19442559       19405635   
749      19287984  [53320.0, 24948.6, 10080.0]  19442581       19405635   
750      19287984  [53320.0, 24948.6, 10080.0]  19442581       19405635   

     confidence1 creator1                    node1_loc  node2_id  \
0              5      NaN  [57828.7, 26647.5, 15880.0]  19272925   
1              5      NaN  [57566.5, 26417.2, 15400.0]  19399331   
2              5      NaN  [57566.5, 26417.2, 15400.0]  19393573   
3              5      NaN  [57631.4, 26483.3, 15480.0]  19272957   
4              5      NaN  [57631.4, 26483.3, 15480.0]  19272958   
..           ...      ...                          ...       ...   
746            5      NaN   [53021.4, 24835.6, 6240.0]  19287942   
747            5      NaN   [53796.1, 24714.6, 6720.0]  19442521   
748            5      NaN   [53825.5, 25027.9, 8680.0]  19417516   
749            5      NaN  [53162.5, 25120.7, 10080.0]  19287985   
750            5      NaN  [53162.5, 25120.7, 10080.0]  19285839   

     target_neuron  confidence2 creator2                    node2_loc  
0         19369066            5      NaN  [58130.2, 26419.5, 16160.0]  
1         19369066            5      NaN  [56921.3, 26085.1, 15200.0]  
2         19363858            5      NaN  [56700.8, 25722.9, 15400.0]  
3         19369066            5      NaN  [58199.2, 26909.2, 15480.0]  
4         19363858            5      NaN  [58188.6, 26643.6, 15480.0]  
..             ...          ...      ...                          ...  
746       19259770            5      NaN   [53111.3, 25091.6, 6240.0]  
747       19259770            5      NaN   [53687.3, 24911.2, 6840.0]  
748       19386927            5      NaN   [53560.8, 24954.0, 8520.0]  
749       19259770            5      NaN  [53588.7, 25058.0, 10080.0]  
750       19258184            5      NaN   [53512.1, 25094.7, 9880.0]  

[751 rows x 12 columns]

In [25]:
#Find connectors below 100nm

# Compute distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_tnt.apply(lambda row: euclidean_distance(row['connector_loc'], row['node2_loc']), axis=1)
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
tnt_filtered_df = connectors_tnt[small_distances]
len(tnt_filtered_df)

4

# FraRobo

In [26]:
pymaid.clear_cache()
frarobo = pymaid.connect_catmaid(server="https://iav-tnt.catmaid.virtualflybrain.org/", project_id=2, api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {tnt.catmaid_version}')
neurons_frarobo = pymaid.find_neurons(annotations='Valdes-Aleman et al. 2021')

# Get connectors of neurons of interest
print(f'Server is running CATMAID version {wt.catmaid_version}')
connectors_frarobo = pymaid.get_connectors_between(neurons_frarobo, neurons_frarobo)
connectors_frarobo 

INFO  : Cached cleared. (pymaid)
INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


INFO  : Found 36 neurons matching the search parameters (pymaid)                
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


connector_id                connector_loc  node1_id  source_neuron  \
0        19272919  [57813.4, 26462.7, 15880.0]  19272920       19243093   
1        19272932  [57285.6, 26135.5, 15400.0]  19272933       19243093   
2        19272932  [57285.6, 26135.5, 15400.0]  19272933       19243093   
3        19272955  [57855.8, 26723.2, 15480.0]  19272956       19243093   
4        19272955  [57855.8, 26723.2, 15480.0]  19272956       19243093   
..            ...                          ...       ...            ...   
746      19287943   [53010.5, 24947.6, 6240.0]  19442479       19405635   
747      19442515   [53847.9, 24936.4, 6720.0]  19442516       19405635   
748      19442558   [53741.4, 24991.1, 8680.0]  19442559       19405635   
749      19287984  [53320.0, 24948.6, 10080.0]  19442581       19405635   
750      19287984  [53320.0, 24948.6, 10080.0]  19442581       19405635   

     confidence1 creator1                    node1_loc  node2_id  \
0              5      NaN  [57828.7, 26647.5, 15880.0]  19272925   
1              5      NaN  [57566.5, 26417.2, 15400.0]  19399331   
2              5      NaN  [57566.5, 26417.2, 15400.0]  19393573   
3              5      NaN  [57631.4, 26483.3, 15480.0]  19272957   
4              5      NaN  [57631.4, 26483.3, 15480.0]  19272958   
..           ...      ...                          ...       ...   
746            5      NaN   [53021.4, 24835.6, 6240.0]  19287942   
747            5      NaN   [53796.1, 24714.6, 6720.0]  19442521   
748            5      NaN   [53825.5, 25027.9, 8680.0]  19417516   
749            5      NaN  [53162.5, 25120.7, 10080.0]  19287985   
750            5      NaN  [53162.5, 25120.7, 10080.0]  19285839   

     target_neuron  confidence2 creator2                    node2_loc  
0         19369066            5      NaN  [58130.2, 26419.5, 16160.0]  
1         19369066            5      NaN  [56921.3, 26085.1, 15200.0]  
2         19363858            5      NaN  [56700.8, 25722.9, 15400.0]  
3         19369066            5      NaN  [58199.2, 26909.2, 15480.0]  
4         19363858            5      NaN  [58188.6, 26643.6, 15480.0]  
..             ...          ...      ...                          ...  
746       19259770            5      NaN   [53111.3, 25091.6, 6240.0]  
747       19259770            5      NaN   [53687.3, 24911.2, 6840.0]  
748       19386927            5      NaN   [53560.8, 24954.0, 8520.0]  
749       19259770            5      NaN  [53588.7, 25058.0, 10080.0]  
750       19258184            5      NaN   [53512.1, 25094.7, 9880.0]  

[751 rows x 12 columns]

In [28]:
#Find connectors below 100nm

# Compute distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_tnt.apply(lambda row: euclidean_distance(row['connector_loc'], row['node2_loc']), axis=1)
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
tnt_filtered_df = connectors_tnt[small_distances]
len(tnt_filtered_df)

24